Задача текстовой классификации.  

Для этой задачи есть соревнование на kaggle https://www.kaggle.com/c/whats-cooking/

Дано:  
    1. Ингридиенты кулинарного рецепта на английском языке.
    2. Название страны происхождения рецепта.
    
Необходимо составить модель, которая будет определять наименование страны по содержимому рецепта.

Загружаю необходимые библиотеки:

In [1]:
import pandas as pd
import numpy as np

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Получаю данные из источника:

In [76]:
data=pd.read_json('train.json')

In [77]:
data.head(10)

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."
5,jamaican,6602,"[plain flour, sugar, butter, eggs, fresh ginge..."
6,spanish,42779,"[olive oil, salt, medium shrimp, pepper, garli..."
7,italian,3735,"[sugar, pistachio nuts, white almond bark, flo..."
8,mexican,16903,"[olive oil, purple onion, fresh pineapple, por..."
9,italian,12734,"[chopped tomatoes, fresh basil, garlic, extra-..."


In [78]:
data.shape

(39774, 3)

In [79]:
data.cuisine.value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

На входе имею 39774 рецептов в 20 категориях. Есть категории с очень большим количеством рецептов и с очень маленьким - это нужно будет учесть.

In [80]:
data.shape

(39774, 3)

Теперь, когда есть контрольные данные, можно пробовать различные модели.

In [81]:
dict_y=data.cuisine.unique()
X=data.ingredients.apply(lambda x: ', '.join(x))
y=[]
for x in data.cuisine:
    y.extend(np.where(dict_y == x)[0])

In [82]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

Пробую модели с TfidfVectorizer и количеством n-gramm от 1 до 2.  
Значение ошибки равна среднему значению ошибки кросс-валидации модели по 5 фолдам.

In [83]:
tfdf_svc_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf', LinearSVC(random_state=123))])

tfdf_log_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf', LogisticRegression(random_state=123))])

tfdf_sgd_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

tfdf_svc_res=cross_val_score(tfdf_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_log_res=cross_val_score(tfdf_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_sgd_res=cross_val_score(tfdf_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("tfdf_svc mean:",np.mean(tfdf_svc_res))
print("tfdf_log mean:",np.mean(tfdf_log_res))
print("tfdf_sgd mean:",np.mean(tfdf_sgd_res))

tfdf_svc mean: 0.7883290862664112
tfdf_log mean: 0.7647959070159182
tfdf_sgd mean: 0.7736210743296008


Попробую те же модели с CountVectorizer

In [84]:
count_svc_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('clf', LinearSVC(random_state=123))])

count_log_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('clf', LogisticRegression(random_state=123))])

count_sgd_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

count_svc_res=cross_val_score(count_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_log_res=cross_val_score(count_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_sgd_res=cross_val_score(count_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("count_svc mean:",np.mean(count_svc_res))
print("count_log mean:",np.mean(count_log_res))
print("count_sgd mean:",np.mean(count_sgd_res))

count_svc mean: 0.7603965381282844
count_log mean: 0.7862923830998303
count_sgd mean: 0.7714075710537001


CountVectorizer с n-gramm от 1 до 3

In [85]:
count_svc_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                     ('clf', LinearSVC(random_state=123))])

count_log_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                     ('clf', LogisticRegression(random_state=123))])

count_sgd_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 3))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

count_svc_res=cross_val_score(count_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_log_res=cross_val_score(count_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_sgd_res=cross_val_score(count_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("count_svc mean:",np.mean(count_svc_res))
print("count_log mean:",np.mean(count_log_res))
print("count_sgd mean:",np.mean(count_sgd_res))

count_svc mean: 0.7687435539709216
count_log mean: 0.784456680192035
count_sgd mean: 0.7673590328741229


TfidfVectorizer с n-gramm от 1 до 3

In [86]:
tfdf_svc_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 3))),
                     ('clf', LinearSVC(random_state=123))])

tfdf_log_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 3))),
                     ('clf', LogisticRegression(random_state=123))])

tfdf_sgd_clf = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 3))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

tfdf_svc_res=cross_val_score(tfdf_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_log_res=cross_val_score(tfdf_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_sgd_res=cross_val_score(tfdf_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("tfdf_svc mean:",np.mean(tfdf_svc_res))
print("tfdf_log mean:",np.mean(tfdf_log_res))
print("tfdf_sgd mean:",np.mean(tfdf_sgd_res))

tfdf_svc mean: 0.7843317378592742
tfdf_log mean: 0.7425955164873553
tfdf_sgd mean: 0.7565747380203578


Попробую стеммер

In [87]:
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/kirill/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [88]:
stemmer = nltk.stem.WordNetLemmatizer()
cnt_analyzer = CountVectorizer().build_analyzer()
tf_analyzer = TfidfVectorizer().build_analyzer()

def cnt_stemmed_words(doc):
    return (stemmer.lemmatize(w) for w in cnt_analyzer(doc))

def tf_stemmed_words(doc):
    return (stemmer.lemmatize(w) for w in tf_analyzer(doc))

In [89]:
tfdf_svc_clf = Pipeline([('vect', TfidfVectorizer(analyzer=tf_stemmed_words,
                                                  ngram_range=(1, 2))),
                     ('clf', LinearSVC(random_state=123))])

tfdf_log_clf = Pipeline([('vect', TfidfVectorizer(analyzer=tf_stemmed_words,
                                                  ngram_range=(1, 2))),
                     ('clf', LogisticRegression(random_state=123))])

tfdf_sgd_clf = Pipeline([('vect', TfidfVectorizer(analyzer=tf_stemmed_words,
                                                  ngram_range=(1, 2))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

tfdf_svc_res=cross_val_score(tfdf_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_log_res=cross_val_score(tfdf_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
tfdf_sgd_res=cross_val_score(tfdf_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("tfdf_svc mean:",np.mean(tfdf_svc_res))
print("tfdf_log mean:",np.mean(tfdf_log_res))
print("tfdf_sgd mean:",np.mean(tfdf_sgd_res))

tfdf_svc mean: 0.7891841349874593
tfdf_log mean: 0.7764115414458194
tfdf_sgd mean: 0.7761090794632606


In [90]:
count_svc_clf = Pipeline([('vect', CountVectorizer(analyzer=cnt_stemmed_words,
                                                   ngram_range=(1, 2))),
                     ('clf', LinearSVC(random_state=123))])

count_log_clf = Pipeline([('vect', CountVectorizer(analyzer=cnt_stemmed_words,
                                                   ngram_range=(1, 2))),
                     ('clf', LogisticRegression(random_state=123))])

count_sgd_clf = Pipeline([('vect', CountVectorizer(analyzer=cnt_stemmed_words,
                                                   ngram_range=(1, 2))),
                     ('clf', SGDClassifier(random_state=123,max_iter=1000,tol=1e-3))])

count_svc_res=cross_val_score(count_svc_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_log_res=cross_val_score(count_log_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)
count_sgd_res=cross_val_score(count_sgd_clf,X,y,cv=5,scoring='accuracy',n_jobs=3)

print("count_svc mean:",np.mean(count_svc_res))
print("count_log mean:",np.mean(count_log_res))
print("count_sgd mean:",np.mean(count_sgd_res))

count_svc mean: 0.7766393503597417
count_log mean: 0.7854135808477019
count_sgd mean: 0.7692968864673335


Как видно, результат стал лучше, но далёк от совершенного.  
Так как этот датасет является частью соревнования Kaggle, я хочу классифицировать их контрольную выборку и получить результат.  
Для начала, я хочу обучить модель на всех данных.

In [91]:
count_log_clf.fit(X,y)
tfdf_svc_clf.fit(X,y)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer=<function tf_stemmed_words at 0x7f91a1c7bd90>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 2), norm='l2',...max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=123, tol=0.0001,
     verbose=0))])

In [92]:
test=pd.read_json('test.json')
X_pred=test.ingredients.apply(lambda x: ', '.join(x))
y_pred=tfdf_svc_clf.predict(X_pred)

In [93]:
sub=pd.read_csv('sample_submission.csv')

In [94]:
y_sub=[]
for x in y_pred:
    y_sub.append(dict_y[x])

In [95]:
sub.cuisine=y_sub

In [96]:
sub.id=test.id

In [97]:
sub.to_csv('submission.csv', index=False)

In [98]:
sub.cuisine.value_counts()

italian         2080
mexican         1620
southern_us     1192
indian           806
chinese          723
french           693
thai             420
cajun_creole     377
japanese         298
greek            281
moroccan         188
korean           187
vietnamese       182
spanish          172
filipino         163
british          154
irish            116
jamaican         105
russian           95
brazilian         92
Name: cuisine, dtype: int64

По результатам проверки на kaggle я получил 78,8% точности, что на 5% ниже результата победителя в этом соревновании.  